<a href="https://colab.research.google.com/github/AmaniUsagi/Covid-19-Trends-Analysis/blob/main/Covid_19_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Library Imports
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
from datetime import date, datetime
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pof
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')

from pandas_profiling import ProfileReport 

Get previous day data because its complete

In [3]:
today = datetime.now()
jana_str = '%s %d, %d' %(date.today().strftime('%b'), today.day-1, today.year)
jana_str

'May 2, 2022'

**Scraping the Web : https://www.worldometers.info/coronavirus/#countries**

In [4]:
url = "https://www.worldometers.info/coronavirus/#countries"

# Opening the url
req = Request(url=url, headers={'user-agent':'Mozilla/5.0'})
webpage = urlopen(req)
page_soup = bs(webpage, 'html.parser')

Querying for spesific tables of data

In [5]:
table = page_soup.findAll('table',{'id': 'main_table_countries_yesterday'})
containers = table[0].findAll('tr',{'style':''})
title = containers[0]

del containers[0]

# Creating list to hold data
all_data = []

clean = True

# Itterate for all countries
for country in containers:
  country_data = []
  country_container = country.findAll('td')

  if country_container[1].text == ' ':
    continue
  for i in range(1, len(country_container)):
    final_feature = country_container[i].text

    # Cleaning the data collected
    if clean :
      if i != 1 and i != len(country_container) -1:
        final_feature = final_feature.replace(',','')

        if final_feature.find('+') != -1:
          final_feature = final_feature.replace('+','')
          final_feature = float(final_feature)

        elif final_feature.find('-') != -1:
          final_feature = final_feature.replace('-','')
          final_feature = float(final_feature)*-1
    
    if final_feature == 'N/A':
      final_feature = 0
    elif final_feature == '' or final_feature == ' ':
      final_feature = -1

    # Appendig the data
    country_data.append(final_feature)
  all_data.append(country_data)

In [6]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace=True, axis = 1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,513861048,259787.0,6263118,1366.0,468306002,703146.0,39291928,40865,65924,803.5,-1,-1,-1,All
1,China,217452,865.0,5092,32.0,192187,4662.0,20173,576,151,4,160000000,111163,1439323776,Asia
2,USA,83170407,40784.0,1021089,122.0,80757424,37094.0,1391894,1550,248601,3052,1006269592,3007793,334554103,North America
3,India,43085166,2664.0,523889,20.0,42541887,2911.0,19390,698,30669,373,838208698,596663,1404828550,Asia
4,Brazil,30460997,6507.0,663657,90.0,29554308,21269.0,243032,8318,141466,3082,63776166,296187,215324039,South America


In [7]:
# header titles
column_labels = ["Country","Total Cases","New Cases","Total Deaths","New Deaths","Total Recovered","New Recovered","Active Cases",
                 "Serious/Critical","Total cases/1M","Deaths/1M","Total Tests","Test/1M","Polulation","Continent"]
df.columns = column_labels
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Polulation,Continent
0,World,513861048,259787.0,6263118,1366.0,468306002,703146.0,39291928,40865,65924,803.5,-1,-1,-1,All
1,China,217452,865.0,5092,32.0,192187,4662.0,20173,576,151,4,160000000,111163,1439323776,Asia
2,USA,83170407,40784.0,1021089,122.0,80757424,37094.0,1391894,1550,248601,3052,1006269592,3007793,334554103,North America
3,India,43085166,2664.0,523889,20.0,42541887,2911.0,19390,698,30669,373,838208698,596663,1404828550,Asia
4,Brazil,30460997,6507.0,663657,90.0,29554308,21269.0,243032,8318,141466,3082,63776166,296187,215324039,South America


Percentage increase in Cases, Deaths and Recovered

In [8]:
# Typecasting to convert the variable data type into a certain data type in order to the operation required to be performed avoid typeerror
for label in df.columns:
  if label != 'Country' and label !='Continent':
    df[label] = pd.to_numeric(df[label])

In [9]:
df["Inc Cases(%)"] = df['New Cases']/df['Total Cases']*100
df["Inc Deaths(%)"] = df['New Deaths']/df['Total Deaths']*100
df["Inc Recovered(%)"] = df['New Recovered']/df['Total Recovered']*100
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Polulation,Continent,Inc Cases(%),Inc Deaths(%),Inc Recovered(%)
0,World,513861048,259787.0,6263118,1366.0,468306002,703146.0,39291928,40865,65924.0,803.5,-1,-1,-1,All,0.050556,0.021810,0.150147
1,China,217452,865.0,5092,32.0,192187,4662.0,20173,576,151.0,4.0,160000000,111163,1439323776,Asia,0.397789,0.628437,2.425762
2,USA,83170407,40784.0,1021089,122.0,80757424,37094.0,1391894,1550,248601.0,3052.0,1006269592,3007793,334554103,North America,0.049037,0.011948,0.045933
3,India,43085166,2664.0,523889,20.0,42541887,2911.0,19390,698,30669.0,373.0,838208698,596663,1404828550,Asia,0.006183,0.003818,0.006843
4,Brazil,30460997,6507.0,663657,90.0,29554308,21269.0,243032,8318,141466.0,3082.0,63776166,296187,215324039,South America,0.021362,0.013561,0.071966


**Exploratory Data Analysis**

In [10]:
# Generic visualisation of the data
cases = df[['Total Recovered', 'Active Cases', 'Total Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ['Type','Total']

cases_df['Percentage'] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df['Virus'] = ['Covid-19' for i in range(len(cases_df))]

fig = px.bar(cases_df, x = 'Virus', y = 'Percentage', color = 'Type', title='Covid-19 Reports', hover_data=['Total'])
fig.show()

In [11]:
cases = df[['New Cases', 'New Recovered', 'New Deaths']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ['Type','Total']

cases_df['Percentage'] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df['Virus'] = ['Covid-19' for i in range(len(cases_df))]

fig = px.bar(cases_df, x = 'Virus', y = 'Percentage', color = 'Type', title='Covid-19 New Cases', hover_data=['Total'])
fig.show()

In [12]:
perinc = np.round(df[['Inc Cases(%)', 'Inc Deaths(%)', 'Inc Recovered(%)']].loc[0], 2)

perinc_df = pd.DataFrame(perinc)
perinc_df.columns = ['Percentage']
fig = go.Figure()
fig.add_trace(go.Bar(x = perinc_df.index, y = perinc_df['Percentage'], marker_color = ['Red', 'Blue', 'Green']))
fig.show()

**Visualisation with respect to the Continents**

In [13]:
continent_df = df.groupby('Continent').sum().drop('All')
continent_df = continent_df.reset_index()
continent_df

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Polulation,Inc Cases(%),Inc Deaths(%),Inc Recovered(%)
0,Africa,11899607,2795.0,253699,-45.0,10026042,2027.0,510412,944,1845473.0,17902.0,144540908,10861647,1395036609,0.398903,-15.349344,0.333617
1,Asia,148104337,88428.0,1424497,299.0,125152477,90199.0,3059113,12645,5248368.0,31346.0,2101073860,81118855,4660758564,13.415717,-5.172665,7.474758
2,Australia/Oceania,7233835,38234.0,10662,29.0,6757379,39847.0,393692,163,1765585.0,6363.0,78030847,15140408,43489018,-48.139442,239.232993,-150.073588
3,Europe,191320119,73658.0,1818982,641.0,170005746,501475.0,9345853,9187,15173302.0,112545.0,2657597993,198617835,748455152,1.641241,-10.441469,5.464384
4,North America,98442482,47760.0,1460270,175.0,94244089,42732.0,2727292,7092,6760011.0,51211.0,1125673724,89153133,597556127,1.057941,-204.270789,0.018272
5,South America,56832702,8803.0,1294760,105.0,50810414,26758.0,539643,10764,1739380.0,31517.0,219640812,12624176,437263279,-0.679263,99.572839,0.243105


In [14]:
def cont_data_visual(vis_list):
  for label in vis_list:
    con_df = continent_df[['Continent', label]]
    con_df['Percentage'] = np.round(100*con_df[label]/np.sum(con_df[label]), 2)
    con_df['Virus'] = ['Covid-19' for i in range(len(con_df))]

    fig = px.bar(con_df, x = 'Virus', y = 'Percentage', color = 'Continent', hover_data=[label])
    fig.update_layout(title = {'text': f'{label}' })
    fig.show()
    gc.collect()

In [15]:
cases_list = ['Total Cases', 'Active Cases', 'New Cases', 'Serious/Critical', 'Total cases/1M']

deaths_list = ['Total Deaths', 'New Deaths', 'Deaths/1M']

recovered_list = ['Total Recovered','New Recovered', 'Inc Recovered(%)']

In [16]:
cont_data_visual(cases_list)

**Vusialization with respect to country**

In [17]:
df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Total cases/1M,Deaths/1M,Total Tests,Test/1M,Polulation,Continent,Inc Cases(%),Inc Deaths(%),Inc Recovered(%)
1,China,217452,865.0,5092,32.0,192187,4662.0,20173,576,151.0,4.0,160000000,111163,1439323776,Asia,0.397789,0.628437,2.425762
2,USA,83170407,40784.0,1021089,122.0,80757424,37094.0,1391894,1550,248601.0,3052.0,1006269592,3007793,334554103,North America,0.049037,0.011948,0.045933
3,India,43085166,2664.0,523889,20.0,42541887,2911.0,19390,698,30669.0,373.0,838208698,596663,1404828550,Asia,0.006183,0.003818,0.006843
4,Brazil,30460997,6507.0,663657,90.0,29554308,21269.0,243032,8318,141466.0,3082.0,63776166,296187,215324039,South America,0.021362,0.013561,0.071966
5,France,28690748,8737.0,146142,180.0,27017667,190730.0,1526939,1557,437774.0,2230.0,266484045,4066113,65537780,Europe,0.030452,0.123168,0.705945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Falkland Islands,128,-1.0,-1,-1.0,0,0.0,0,-1,34925.0,-1.0,8632,2355252,3665,South America,-0.781250,100.000000,NaN
217,Marshall Islands,17,-1.0,-1,-1.0,14,-1.0,3,-1,284.0,-1.0,-1,-1,59921,Australia/Oceania,-5.882353,100.000000,-7.142857
218,Niue,9,-1.0,-1,-1.0,7,-1.0,2,-1,5468.0,-1.0,-1,-1,1646,Australia/Oceania,-11.111111,100.000000,-14.285714
219,Micronesia,7,-1.0,-1,-1.0,1,-1.0,6,-1,60.0,-1.0,-1,-1,117229,Australia/Oceania,-14.285714,100.000000,-100.000000


Check The Top 5 Affected

In [21]:
LOOK_AT = 10
country = country_df.columns[1:14]
fig = go.Figure()
c = 0
for i in country_df.index:
  if c < LOOK_AT:
    fig.add_trace(go.Bar(name = country_df['Country'][i], x = country, y=country_df.loc[i][1:14]))
  else:
    break
  c +=1
fig.update_layout(title = {'text': f'Top {LOOK_AT} Countries Affected'}, yaxis_type = 'log')
fig.show()